## Requirement
pip install facenet-pytorch

In [ ]:
from facenet_pytorch import MTCNN
from PIL import Image
import torch
from imutils.video import FileVideoStream
import cv2
import time
import glob
from tqdm.notebook import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Face Detection - FastMTCNN 

In [ ]:
class FastMTCNN(object):
    """Fast MTCNN implementation."""
    
    def __init__(self, stride, resize=1, *args, **kwargs):
        """Constructor for FastMTCNN class.
        
        Arguments:
            stride (int): The detection stride. Faces will be detected every `stride` frames
                and remembered for `stride-1` frames.
        
        Keyword arguments:
            resize (float): Fractional frame scaling. [default: {1}]
            *args: Arguments to pass to the MTCNN constructor. See help(MTCNN).
            **kwargs: Keyword arguments to pass to the MTCNN constructor. See help(MTCNN).
        """
        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)
        
    def __call__(self, frames):
        """Detect faces in frames using strided MTCNN."""
        if self.resize != 1:
            frames = [
                cv2.resize(f, (int(f.shape[1] * self.resize), int(f.shape[0] * self.resize)))
                    for f in frames
            ]
            
        boxes, probs = self.mtcnn.detect(frames[::self.stride])

        faces = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            else:
                for box in boxes[box_ind]:
                    box = [int(b) for b in box]

                    image_rgb = frame[box[1]:box[3], box[0]:box[2]]
                    if (len(image_rgb) > 0) and (image_rgb.shape[0] > 10) and (image_rgb.shape[1] > 10):
                        faces.append(image_rgb)
                        ts = time.time()


                        img_path = '/home/umit/xDataset/Sentinel-img/train-real/train-real-%f.jpeg' %ts

                        #image_rgb = cv2.resize(image_rgb, (256,256))
                        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
                        cv2.imwrite(img_path, image_bgr)
                
        return faces

In [ ]:
# help(MTCNN)

In [ ]:
fast_mtcnn = FastMTCNN(
    stride=1,
    resize=1,
    margin=50,
    min_face_size=100, #default =20
    thresholds=[0.6, 0.7, 0.7],
    factor=0.7, # default = 0.709
    post_process=True,
    select_largest=True,
    keep_all=True,
    device=device
)

## VIDEO 

In [ ]:
filenames_video = glob.glob('../*.mp4')

len(filenames_video)
jump = 30 # all frame jump=1
def run_video_detection(fast_mtcnn, filenames_video):
    
    frames = []
    frames_processed = 0
    faces_detected = 0
    batch_size = 60
    
    for filename in tqdm(filenames_video):
        
        v_cap = FileVideoStream(filename).start()
        v_len = int(v_cap.stream.get(cv2.CAP_PROP_FRAME_COUNT))
        print("vlen = "+str(v_len))

        for j in range(0,v_len):

            frame = v_cap.read()
            
            if j%jump==0 or j == v_len - 1:
                
                #frame = cv2.flip(cv2.transpose(frame), flipCode=1)
                
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame)

                if len(frames) >= batch_size or j == v_len - 1:

                    faces = fast_mtcnn(frames)

                    frames_processed += len(frames)
                    faces_detected += len(faces)

                    frames = []
                    faces = []
                
        v_cap.stop()
        
        print("frames_processed = "+str(frames_processed))
        print("faces_detected = "+str(faces_detected))

run_video_detection(fast_mtcnn, filenames_video)

## IMAGE

In [ ]:
image = True
if image == True:
    
    filenames_image = glob.glob('../*.jpg')
    def run_image_detection(fast_mtcnn, filenames_image):

        images = []
        images_processed = 0
        faces_detected = 0

        for filename in tqdm(filenames_image):

            image = cv2.imread(filename)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(image)

            face = fast_mtcnn(images)

            images_processed += len(images)
            faces_detected += len(face)

            images = []

        print("images_processed = "+str(images_processed))
        print("faces_detected = "+str(faces_detected))

    run_image_detection(fast_mtcnn, filenames_image)